# RCS Benchmark

In [1]:
low = 8
high = 64
samples = 10

In [2]:
import time
import random
import math

from pyqrack import QrackSimulator

## PyQrack

In [3]:
def bench_qrack(n):
    # This is basically a "quantum volume" (random) circuit.
    start = time.perf_counter()

    sim = QrackSimulator(n)

    lcv_range = range(n)
    all_bits = [list(lcv_range)]

    for _ in lcv_range:
        # Single-qubit gates
        for i in lcv_range:
            sim.u(i, random.uniform(0, 2 * math.pi), random.uniform(0, 2 * math.pi), random.uniform(0, 2 * math.pi))

        # 2-qubit couplers
        unused_bits = all_bits.copy()
        random.shuffle(unused_bits)
        while len(unused_bits) > 1:
            sim.mcx([unused_bits.pop()], unused_bits.pop())

    # Terminal measurement
    # print(sim.m_all())
    sim.m_all()

    return time.perf_counter() - start

# Make sure the OpenCL environment is initialized before timing.
# (You probably also want to precompile OpenCL kernels with the `qrack_cl_compile` utility, in general.)
bench_qrack(1)

qrack_results = {}
for n in range(low, high + 1):
    width_results = []
        
    # Run the benchmarks
    for i in range(samples):
        width_results.append(bench_qrack(n))

    qrack_results[n] = sum(width_results) / samples
    print(n, ": ", qrack_results[n], " seconds")

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
8 :  0.0002663639003003482  seconds
9 :  0.0003225471002224367  seconds
10 :  0.00038207889956538567  seconds
11 :  0.000450291701417882  seconds
12 :  0.0005365994984458666  seconds
13 :  0.0005974229999992531  seconds
14 :  0.0006984417988860514  seconds
15 :  0.0007879812008468434  seconds
16 :  0.0008911625998734962  seconds
17 :  0.0009917194976878819  seconds
18 :  0.0011141800001496448  seconds
19 :  0.0012114302007830702  seconds
20 :  0.0014201400001184084  seconds
21 :  0.0015007636990048922  seconds
22 :  0.0015927180982544086  seconds
23 :  0.0017340862999844831  seconds
24 :  0.0018029551014478785  seconds
25 :  0.001971206598682329  seconds
26 :  0.002291568700456992  seconds
27 :  0.0023964479005371686  seconds
28 :  0.0030285485001513733  seconds
29 :  0.0032426411991764324  seconds
30 :  0.003566655299800914  seconds
31 :  0.0036772728999494575  seconds
32 :  0.00401670